Здесь должен быть текст описания

In [1]:
import pandas as pd
import re
from scipy.stats import chi2_contingency
from math import log, exp, sqrt
import gdown
import os

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)

In [3]:
file_id = '1BM7QmO4GoR5YxlMPfM5DLyrxBm3mn3oB'
dataset_link = f'https://drive.google.com/uc?id={file_id}'
output_file = 'after_eda_dataset.zip'

In [4]:
if not os.path.exists(output_file):
    gdown.download(dataset_link, output_file, quiet=False)
    print(f"Файл {output_file} скачан.")
else:
    print(f"Файл {output_file} уже существует.")

Файл after_eda_dataset.zip уже существует.


## Гипотеза 1

- **H1**: Партии, где стороны рокировались в разные фланги, заканчиваются чаще результативно (win/lose), чем партии с одинаковой рокировкой.
- **H0**: Доли результативных партий в группах рокировавшихся в одну сторону и в разные стороны равны.

In [5]:
df = pd.read_csv(output_file, compression='zip')

In [6]:
# Убираем игры, где кол-во ходов было меньше 10.
# Т.е. игры по какой-либо причине закончившиеся слишком быстро
# Стоило бы это сделать на этапе EDA еще
df = df[(df["Num_Moves"] >= 10) & (df['Game_Duration'] >= 15)]

In [7]:
# Очищаем шахматные ходы (moves) от лишних знаков
# И создаем колонку - white_castle и black_castle - какую рокировку делали белые и черные.
def clean_move(move_str):
    return move_str.rstrip("+#?!")


def define_castle(moves_str, color):
    cl_df = re.sub(r"\{[^}]*\}", " ", moves_str)
    cl_df = re.sub(r"\d+\.", " ", cl_df)
    cl_df = re.sub(r"\s*(1-0|0-1|1/2-1/2)\s*$", " ", cl_df)
    cl_df = " ".join(cl_df.split())

    moves = cl_df.split(" ")
    if color == "white":
        rel_moves = [clean_move(m) for i, m in enumerate(moves) if i % 2 == 0]
    else:
        rel_moves = [clean_move(m) for i, m in enumerate(moves) if i % 2 == 1]

    for move in rel_moves:
        if move.startswith("O-O-O"):
            return "O-O-O"
        elif move.startswith("O-O"):
            return "O-O"
    return pd.NA


df["white_castle"] = df["Moves"].apply(lambda moves: define_castle(moves, "white"))
df["white_castle"].value_counts(dropna=False)

white_castle
O-O      84686
O-O-O    14173
<NA>     12978
Name: count, dtype: int64

In [8]:
df["black_castle"] = df["Moves"].apply(lambda moves: define_castle(moves, "black"))
df["black_castle"].value_counts(dropna=False)

black_castle
O-O      85909
<NA>     17573
O-O-O     8355
Name: count, dtype: int64

In [9]:
# Сравниваем рокировку белых и черных
def compare_castle(row):
    white, black = row["white_castle"], row["black_castle"]

    if pd.isna(white) or pd.isna(black):
        return "None"

    return "Same" if white == black else "Opposite"


df["castle_type"] = df.apply(compare_castle, axis=1)
df["castle_type"].value_counts()

castle_type
Same        73631
None        25918
Opposite    12288
Name: count, dtype: int64

In [10]:
# Создаем колонку - игра была результативная или ничейная
df["result_game"] = df["Winner"].apply(
    lambda x: "draw" if x == "draw" else "has_result")

Почему вы выбрали именно этот тест для проверки?!

In [11]:
records = []

# Проходимся по каждому speed_type 
# -> оставляем только игры, где были рокировки у обеих сторон
# -> создаем сопряженную таблицу с булевыми значениями
# -> вычисляем OR - отношение шансов
# -> проводим тест chi2, т.к. работаем с частотностью
# -> вычисляем доверительный интервал
# -> добавляем все результаты в один массив и создаем красивую таблицу для вывода
for speed_type, sub_df in df.groupby("Speed"):
    sub_df = sub_df[sub_df["castle_type"].isin(["Opposite", "Same"])]

    table = pd.crosstab(
        sub_df["castle_type"],
        sub_df["result_game"] == "has_result"
    ).reindex(
        index=["Opposite", "Same"],
        columns=[True, False],
        fill_value=0
    )

    opp_has_result, opp_draw = table.loc["Opposite"]
    same_has_result, same_draw = table.loc["Same"]

    odds_ratio = (opp_has_result * same_draw) / (opp_draw * same_has_result)
    se_log_or = sqrt(
        1/opp_has_result + 1/opp_draw + 1/same_has_result + 1/same_draw
    )
    ci_low = exp(log(odds_ratio) - 1.96 * se_log_or)
    ci_high = exp(log(odds_ratio) + 1.96 * se_log_or)

    _, p_value, _, _ = chi2_contingency(table, correction=False)

    records.append({
        "Speed": speed_type,
        "Opp_has_result": opp_has_result,
        "Opp_total":    opp_has_result + opp_draw,
        "Same_has_result": same_has_result,
        "Same_total":    same_has_result + same_draw,
        "OR": round(odds_ratio, 3),
        "CI_low": round(ci_low, 3),
        "CI_high": round(ci_high, 3),
        "p_value": p_value
    })

summary = pd.DataFrame(records)
summary

,Speed,Opp_has_result,Opp_total,Same_has_result,Same_total,OR,CI_low,CI_high,p_value
0,blitz,3293,3571,18947,21227,1.425,1.252,1.623,7.726398e-08
1,bullet,2609,2747,15520,16724,1.467,1.224,1.758,3.024523e-05
2,classical,2335,2572,12878,14848,1.507,1.308,1.737,1.161354e-08
3,rapid,2667,2913,16107,18449,1.576,1.374,1.809,6.480513e-11
4,ultraBullet,474,485,2323,2383,1.113,0.581,2.133,7.469061e-01


Т.к. p_value < 0.05, то мы мы можем отвергнуть нулевую гипотезу для всех типов Speed, кроме Ultrabullet.
Значит 

Т.к. OR > 1, значит, что шансы получить решающий результат при разносторонних рокировках выше (opposite), чем при односторонних (same).

## Гипотеза 2